# IronHacks Submission Notebook

## Background
---

Write a brief summary about goal of this notebook and provide some insight to the reader about how read through your document.

## Setup
---

In [ ]:

%logstop
%logstart -t -r -q ipython_command_log.py global

#- IRONHACKS RESEARCH TRACKING CODE
#----------------------------------
# The following code is used to help our research team understand how you 
# our notebook environment. We do not collect any personal information with
# the following code, it is used to measure when and how often you work on
# your submission files.

import os
from datetime import datetime
import IPython.core.history as history

ha = history.HistoryAccessor()
ha_tail = ha.get_tail(1)
ha_cmd = next(ha_tail)
session_id = str(ha_cmd[0])
command_id = str(ha_cmd[1])
timestamp = datetime.utcnow().isoformat()
history_line = ','.join([session_id, command_id, timestamp]) + '\n'
logfile = open(os.environ['HOME']+'/ipython_session_log.csv', 'a')
logfile.write(history_line)
logfile.close()

In [ ]:


#- INSTALL ADDITIONAL LIBRARIES IF REQUIRED
#------------------------------------------
# This is normally not required. The hub environment comes preinstaled with 
# many packages that you can already use without setup. In case there is some
# other library you would like to use that isn't on the list you run this command
# once to install them.  If it is already installed this command has no effect.

!python3 -m pip install pandas

### Imports

In [ ]:
#- IMPORT THE LIBRARIES YOU WILL USE
#------------------------------------------
# You only need to import packages one time per notebook session. To keep your
# notebook clean and organized you can handle all imports at the top of your file.
# The following are included for example purposed, feel free to modify or delete 
# anything in this section.

import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics

## Classes and Functions
---

In [ ]:
#- DEFINE YOUR CLASSES AND FUNCTIONS 
#-----------------------------------
# This is not required, but is helpful in keeping your notebook organized. 
# You can use the following cell or several cells to define your functions
# and classes to keep them separate from your analysis or results code.
# In general it useful to define your methods in a separate cell from where
# it is run.

def example_function():
    print('Hello World')

In [ ]:
# CONFIGURE THE BIGQUERY SETTINGS

BIGQUERY_PROJECT = 'ironhacks-covid19-data'
BIGQUERY_KEYPATH = 'key.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = BIGQUERY_KEYPATH
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [ ]:
query = """
SELECT *
FROM ironhacks-covid19-data.ironhacks_covid19_competition.weekly_patterns
"""

query_job = bigquery_client.query(query)
data = query_job.to_dataframe()
data.head()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:441: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,poi_id,location_name,latitude,longitude,city,postal_code,brands,top_category,NAICS,week_number,date_start,raw_visit_counts,visits_concentration,poi_cbg,distance_from_home,median_dwell
0,6a2d40f8-c477-49b6-a5d5-69c31f001afc,Auto Express,40.367561,-86.868565,Lafayette,47909,None,None,None,11,2020-03-16,23,510.40,181570016001,5834.0,109.0
1,49636347-5a9d-4bf3-9d9a-c21a4ba7d8f1,Del Real Auto Connection,40.438210,-86.859945,Lafayette,47904,None,None,None,11,2020-03-16,9,1111.11,181570003001,4914.0,21.0
2,d98b37cd-ed64-456c-b580-c2c299c9f6d6,Bear's Auto Sales,40.409919,-86.869882,Lafayette,47904,None,None,None,11,2020-03-16,11,1074.38,181570012001,2982.0,40.0
3,efb4f5e7-28c2-47be-9d47-254280b8458e,Gta Auto Sales,40.440608,-86.877960,Lafayette,47904,None,None,None,11,2020-03-16,2,5000.00,181570001002,0.0,171.0
4,16f4e696-2065-4854-8ba6-a8cab2b67bf7,Steve Firestone Auto Sales,40.439060,-86.880571,Lafayette,47904,None,None,None,11,2020-03-16,22,495.87,181570001002,2809.0,117.0


In [ ]:
poi_names = np.unique(data['poi_id'].to_numpy())
raw_rows = data['poi_id'].to_numpy()
poi_id = []
for r in raw_rows:    
    poi_id.append(np.where(poi_names==r)[0][0])

In [ ]:
raw_counts=data[['raw_visit_counts']].to_numpy()
week_number=data[['week_number']].to_numpy()

week_start = 11
number_poi = len(poi_names)
number_weeks = 44-week_start
visit_mat = np.zeros((number_poi,number_weeks))
for i in np.arange(len(raw_rows)):
    visit_mat[poi_id[i],week_number[i]-week_start]=raw_counts[i]

In [ ]:
# linear regression
window_size = 14
X=np.ones((number_poi*(number_weeks-window_size),window_size+1))
y=np.zeros((number_poi*(number_weeks-window_size),1))
X_pred = np.ones((number_poi,window_size+1))
i = 0
i_poi = 0
for visit_ in visit_mat:
    for j in np.arange(44-week_start-window_size):
        X[i,:-1]=visit_[j:(j+window_size)]
        y[i] = visit_[(j+window_size)]
        i = i + 1
    j = j + 1
    X_pred[i_poi,:-1] = visit_[j:(j+window_size)]
    i_poi = i_poi + 1
    

In [ ]:
visit_[(j+window_size)]

24.0

In [ ]:
visit_[j:(j+window_size)]

array([ 4., 10., 17., 18.,  9.,  9., 11., 14., 23., 10.,  7.,  9., 15.,
       10.])

In [ ]:
visit_

array([20.,  8.,  9., 10.,  9.,  3.,  6.,  0.,  0., 14., 15., 11., 12.,
       10., 12.,  8.,  9., 15.,  4., 10., 17., 18.,  9.,  9., 11., 14.,
       23., 10.,  7.,  9., 15., 10., 24.])

In [ ]:
X

array([[13., 11.,  5., ...,  1.,  0.,  1.],
       [11.,  5.,  5., ...,  0.,  0.,  1.],
       [ 5.,  5.,  5., ...,  0.,  2.,  1.],
       ...,
       [ 9., 15.,  4., ...,  7.,  9.,  1.],
       [15.,  4., 10., ...,  9., 15.,  1.],
       [ 4., 10., 17., ..., 15., 10.,  1.]])

In [ ]:
y

array([[ 0.],
       [ 2.],
       [10.],
       ...,
       [15.],
       [10.],
       [24.]])

In [ ]:
X_pred

array([[  2.,  11.,  11., ...,   2.,   7.,   1.],
       [ 23.,  13.,   7., ...,  37.,   8.,   1.],
       [162., 181., 191., ..., 186., 199.,   1.],
       ...,
       [ 58., 125., 167., ..., 129., 144.,   1.],
       [ 32.,  43.,  52., ...,   5.,   9.,   1.],
       [ 10.,  17.,  18., ...,  10.,  24.,   1.]])

In [ ]:
visit_[j:(j+window_size)]

array([15.,  4., 10., 17., 18.,  9.,  9., 11., 14., 23., 10.,  7.,  9.,
       15.])

In [ ]:
coefs=np.matmul(np.linalg.inv(np.matmul(X.transpose(),X)),np.matmul(X.transpose(),y))
coefs

array([[-0.06645172],
       [ 0.02998943],
       [-0.02052899],
       [ 0.05837486],
       [ 0.05167519],
       [-0.04378785],
       [ 0.12189246],
       [-0.1263129 ],
       [ 0.02157328],
       [ 0.00343472],
       [-0.02438314],
       [ 0.08986623],
       [-0.11252987],
       [ 1.00099629],
       [ 3.17919007]])

In [ ]:
len(poi_names)

1804

In [ ]:
y_pred=np.matmul(X_pred,coefs)
len(y_pred)

1804

## Analysis
---

In [ ]:
example_function()

## Results
---

This section may not be necessary for you individual notebook, but if you've created nice charts or tables you can present them together with some explanation in section. In addition if you have any comments or thoughts about what you would like to do or items that are still in progress you can summarize them at the bottom of the page.

In [ ]:
## This can also be a good place for you to cleanup any input/output and export your results to a file.

In [ ]:
import pandas as pd 
df=pd.DataFrame(y_pred)
df['poi_id']=poi_names
df=df.rename(columns={0: "raw_visit_counts"})

df=df[['poi_id','raw_visit_counts']]
df.to_csv('submission_prediction_output.csv',index=False)